* Erstelle eines bereinigten Datensatzes aus einer Webseite

Prompt:

Schreibe einen Python Code, der folgende Funktionen erfüllt:
- Extrahieren aus einer Webseite alle Sätze
- Extrahiere auch aus allen verlinkten Seiten aus der Hauptseite innerhalb der Domäne
- max 10 links probieren
- die Sätze sollen forbereitet werden zur embedding verarbeitung mit openai
- Textblöcke säubern und korrigieren
- Entfernen von Zeilenumbrüchen und Leerzeichen am Anfang und Ende
- nur sätze nehmen die mindestens 10 Wörter hat
- es soll als csv gespeichert werden
- der code soll produktionsreif sein
- der code soll eine hohe qualität aufweisen

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import string

def clean_text(text):
    # Entferne Zeilenumbrüche und Leerzeichen am Anfang und Ende
    text = text.strip().replace('\n', ' ')
    # Entferne führende und nachfolgende Satzzeichen
    text = text.strip(string.punctuation)
    return text

def extract_sentences(url, max_links=10):
    sentences = []
    visited_links = set()
    visited_links.add(url)

    def process_page(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        text_blocks = soup.find_all('p')  # Annahme: Text in Absätzen (<p>-Tags) auf der Seite

        for block in text_blocks:
            text = block.get_text()
            # Bereinige und korrigiere den Text
            text = clean_text(text)
            # Teile den Text in Sätze auf
            sentences.extend(re.split(r'[.!?]+', text))

    def process_links(url, depth=0):
        if depth >= max_links:
            return

        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')  # Annahme: Links in <a>-Tags auf der Seite

        for link in links:
            href = link.get('href')
            if href and href.startswith('/') and href not in visited_links:
                visited_links.add(href)
                new_url = url + href
                process_page(new_url)
                process_links(new_url, depth + 1)

    process_page(url)
    process_links(url)

    # Filtere Sätze mit mindestens 10 Wörtern
    sentences = [sentence.strip() for sentence in sentences if len(sentence.split()) >= 10]
    return sentences

def prepare_sentences(sentences):
    # Vorbereitung der Sätze für OpenAI Embedding-Verarbeitung
    # Implementiere entsprechende Vorbereitungsschritte basierend auf den Anforderungen

    prepared_sentences = []

    for sentence in sentences:
        # Füge hier den Code für die Vorbereitung der Sätze ein

        prepared_sentences.append(sentence)  # Beispiel: Füge den unveränderten Satz hinzu

    return prepared_sentences

def save_to_csv(sentences, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Sentence'])
        writer.writerows([[sentence] for sentence in sentences])

# Beispielaufruf
url = 'https://www.codersunlimited.com'  # URL der Webseite, von der aus die Sätze extrahiert werden sollen
sentences = extract_sentences(url, max_links=10)
prepared_sentences = prepare_sentences(sentences)
save_to_csv(prepared_sentences, 'sentences.csv')


Cannot process https://playwright.dev/python/docs/api/class-test
Cannot process https://playwright.dev/python/docs/api/class-test
Cannot process https://playwright.dev/java/docs/api/class-test
Cannot process https://playwright.dev/java/docs/api/class-test
Cannot process https://playwright.dev/dotnet/docs/api/class-test
Cannot process https://playwright.dev/dotnet/docs/api/class-test
